# gensim实现抽取文档主题

In [3]:
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()

# 设定分词及清理停用词函数
# 熟悉Python的可以使用 open('stopWord.txt').readlines（） 获取停用词list，效率更高
import pandas as pd
stoplist = list(pd.read_csv('C:/TMdata/stopwords.txt',names=['w'],sep='aaa',encoding='utf-8',engine='python').w)
import jieba 
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) if w not in stoplist and len(w) > 1 ] 

filelist = [m_cut(w) for w in df.content]


# 生成文档对应的字典和bow稀疏向量
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist] # 仍为list in list  

tfidf_model = models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
corpus_tfidf  

from gensim.models.ldamodel import LdaModel

# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus , id2word = dictionary ,num_topics = 10, passes = 10) 
ldamodel.print_topics()#打印出10个主题的前十个相关词



Wall time: 3.7 s


[(0,
  '0.016*"改革" + 0.015*"农业" + 0.014*"实践" + 0.014*"水价" + 0.012*"专业" + 0.011*"教学" + 0.009*"学生" + 0.007*"综合" + 0.005*"地下水" + 0.005*"工程"'),
 (1,
  '0.018*"消毒" + 0.018*"合格率" + 0.017*"医疗机构" + 0.012*"监测" + 0.009*"灭菌" + 0.007*"合格" + 0.006*"医院" + 0.006*"紫外线" + 0.005*"样本" + 0.005*"2016"'),
 (2,
  '0.012*"贫困" + 0.011*"模型" + 0.009*"农村" + 0.008*"算法" + 0.007*"耕地" + 0.006*"数据" + 0.006*"研究" + 0.005*"方法" + 0.005*"发展" + 0.005*"金融"'),
 (3,
  '0.016*"创业" + 0.013*"发展" + 0.012*"创新" + 0.011*"大学生" + 0.010*"教育" + 0.009*"高校" + 0.009*"教学" + 0.009*"农业" + 0.008*"学生" + 0.008*"企业"'),
 (4,
  '0.015*"执业" + 0.014*"医院" + 0.012*"项目" + 0.011*"多点" + 0.011*"医师" + 0.009*"医生" + 0.009*"影像学" + 0.008*"单位" + 0.008*"研究" + 0.007*"牵头"'),
 (5,
  '0.024*"休闲" + 0.023*"巢湖" + 0.021*"农业" + 0.013*"发展" + 0.011*"产品" + 0.011*"旅游" + 0.005*"经营" + 0.004*"湿地" + 0.004*"建设" + 0.004*"文化"'),
 (6,
  '0.023*"数据" + 0.019*"文本" + 0.010*"缺陷" + 0.010*"挖掘" + 0.009*"电力" + 0.009*"技术" + 0.008*"分析" + 0.008*"数据挖掘" + 0.006*"电影" + 0.006*"企业"'),
 (7,
  '0.025*"医

# gensim抽取文档主题并计算文档相似度

In [ ]:
from MyDataFrame import MyDataFrame
mdf = MyDataFrame()
df = mdf.new_DataFrame()

# 设定分词及清理停用词函数
# 熟悉Python的可以使用 open('stopWord.txt').readlines（） 获取停用词list，效率更高
import pandas as pd
stoplist = list(pd.read_csv('C:/TMdata/stopwords.txt',names=['w'],sep='aaa',encoding='utf-8',engine='python').w)
import jieba 
def m_cut(intxt):
    return [ w for w in jieba.cut(intxt) if w not in stoplist and len(w) > 1 ] 

filelist = [m_cut(w) for w in df.content]


# 生成文档对应的字典和bow稀疏向量
from gensim import corpora, models  

dictionary = corpora.Dictionary(filelist)  
corpus = [dictionary.doc2bow(text) for text in filelist] # 仍为list in list  

tfidf_model = models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
corpus_tfidf  

from gensim.models.ldamodel import LdaModel

# 列出所消耗的时间备查
%time ldamodel = LdaModel(corpus_tfidf, id2word = dictionary,num_topics = 5, passes = 10) 


from gensim import similarities
simmtx = similarities.MatrixSimilarity(corpus_tfidf) # 使用的矩阵种类需要和拟合模型时相同
simmtx
print(simmtx.index[:2])

# 使用gensim的LDA拟合结果进行演示
query = df.content[0] 
query_bow = dictionary.doc2bow(m_cut(query))
query_tfidf = tfidf_model[query_bow]
lda_vec = ldamodel[query_tfidf] # 转换为lda模型下的向量
sims = simmtx[lda_vec] # 进行矩阵内向量和所提供向量的余弦相似度查询
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims